In [100]:
import pandas as pd
import numpy as np
import string

from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [122]:
train = pd.DataFrame()
labels = pd.DataFrame()
path_file = "C:/Users/natiami/Downloads/Neuro/Project/traindata/A01TInstances.csv"
path_label = "C:/Users/natiami/Downloads/Neuro/Project/traindata/A01TLabels.csv"

for i in range(9):
    print("Path ", path_file)
    
    feature = pd.read_csv(path_file, header=None)
    train = train.append(feature)
    
    label = pd.read_csv(path_label, header=None)
    labels = labels.append(label)

    path_file = path_file.replace(str(i+1), str(i+2))
    path_label = path_label.replace(str(i+1), str(i+2))
    
    # Print updated data dimensions
    print(train.shape)

print("Final data shapes")
print("Data: ", train.shape)
print("Labels: ", labels.shape)

Path  C:/Users/natiami/Downloads/Neuro/Project/traindata/A01TInstances.csv
(672528, 25)
Path  C:/Users/natiami/Downloads/Neuro/Project/traindata/A02TInstances.csv
(1349697, 25)
Path  C:/Users/natiami/Downloads/Neuro/Project/traindata/A03TInstances.csv
(2010227, 25)
Path  C:/Users/natiami/Downloads/Neuro/Project/traindata/A04TInstances.csv
(2611142, 25)
Path  C:/Users/natiami/Downloads/Neuro/Project/traindata/A05TInstances.csv
(3297262, 25)
Path  C:/Users/natiami/Downloads/Neuro/Project/traindata/A06TInstances.csv
(3976242, 25)
Path  C:/Users/natiami/Downloads/Neuro/Project/traindata/A07TInstances.csv
(4657313, 25)
Path  C:/Users/natiami/Downloads/Neuro/Project/traindata/A08TInstances.csv
(5332583, 25)
Path  C:/Users/natiami/Downloads/Neuro/Project/traindata/A09TInstances.csv
(6005911, 25)
Final data shapes
Data:  (6005911, 25)
Labels:  (5552, 3)


In [127]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-21.142578,-23.681641,-21.484375,-25.146484,-25.732422,-27.929688,-14.550781,-22.509766,-25.439453,-28.710938,...,-30.615234,-29.638672,-26.660156,-20.947266,-25.439453,-24.902344,-19.677734,31.738281,-13.671875,-43.945312
1,-21.923828,-23.925781,-24.316406,-25.341797,-26.074219,-25.244141,-16.064453,-22.753906,-28.320312,-29.248047,...,-31.054688,-30.712891,-26.123047,-22.998047,-27.099609,-26.367188,-23.339844,22.460938,-13.183594,-40.527344
2,-15.625000,-19.726562,-18.847656,-21.582031,-20.751953,-21.728516,-12.451172,-20.410156,-23.730469,-25.927734,...,-28.759766,-28.417969,-24.462891,-21.386719,-25.097656,-24.267578,-21.484375,33.691406,-8.300781,-38.574219
3,-16.699219,-13.720703,-20.361328,-20.849609,-24.169922,-22.021484,-10.986328,-16.357422,-25.585938,-24.951172,...,-27.783203,-28.857422,-25.537109,-19.335938,-24.316406,-24.511719,-17.919922,41.992188,-7.324219,-38.574219
4,-19.335938,-17.626953,-20.410156,-26.464844,-27.392578,-29.882812,-9.277344,-18.359375,-24.609375,-30.517578,...,-31.884766,-33.740234,-32.666016,-18.554688,-26.611328,-28.466797,-18.408203,37.109375,-15.136719,-49.804688


In [128]:
label_results = [0] * len(train) # make it work for every file

# Extract the labels for each instance
for index, row in labels.iterrows():
    label = row[0]
    row_nr = row[1]
    duration = row[2]
    if(label == 769 or label == 770 or label == 771 or label == 772):
        duration = 1000
    for i in range(duration):
        current_label = label_results[row_nr + i - 1]
        if label == 769 or label == 770 or label == 771 or label == 772:
            label_results[row_nr + i - 1] = label
        elif current_label != 769 and current_label != 770 and current_label != 771 and current_label != 772:
            label_results[row_nr + i - 1] = 1

# Merge the data
se = pd.Series(label_results)
train['label'] = se.values

In [164]:
# eliminate other classes from data
train = train.dropna()

class1 = train[train.label==769]
class2 = train[train.label==770]
class3 = train[train.label==771]
class4 = train[train.label==772]

result = class1.append(class2)
result = result.append(class3)
result = result.append(class4)
result = result.reset_index() 

result = result.dropna()

In [165]:
# remove extra electrodes
df = result.drop(['index', 22, 23, 24], axis=1)
df = df.reset_index()

In [170]:
# TODO this needs to be reviewed if numbers are correct 
# because in the end it gives the same number of instances for all file as for 1 file

# spectral analyses
last_index = 0
label_vector = [] # train labels
fft_array_rows = [] # train data prepared

for i in range(288):
    current_matrix = df[last_index:last_index+1000]
    
    fft_array_columns = []
    label_vector.append(current_matrix["label"][last_index])
    last_index = last_index + 1000

    for i in range(22):
        fft_array_columns.append(np.fft.fft(current_matrix[i],1)[0])
        
    fft_array_rows.append(fft_array_columns)
        
df_abs = np.abs(fft_array_rows) 

In [171]:
data_x = pd.DataFrame(df_abs) # Instances of the data
data_y = pd.DataFrame(label_vector) # Labels of the data

In [172]:
# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=0)

In [173]:
# Check on NaN values
np.any(np.isnan(X_train))

False

In [174]:
print(X_train.shape)
print(len(y_train))

(230, 22)
230


In [175]:
# Train the LDA model
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train, y_train)

C:\Users\natiami\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [176]:
# Accuracy of prediction on train data
y_pred = lda_clf.predict(X_train)
print("Accuracy on train data ", 100*accuracy_score(y_pred, y_train ))

Accuracy on train data  62.1739130435


In [177]:
# Accuracy of prediction on test data
y_pred = lda_clf.predict(X_test)
print("Accuracy on test data ", 100*accuracy_score(y_pred, y_test ))

Accuracy on test data  53.4482758621


In [156]:
# Precision  on test data
df = {'Precision': precision_score(y_test.values, y_pred, average=None)}
df = pd.DataFrame(data=df)
df

,Precision
0,0.535714
1,0.548387


In [157]:
conf_mat = confusion_matrix(y_test, y_pred, labels=[770,769,771,772])
pd.DataFrame(conf_mat)

,0,1,2,3
0,17,26,0,0
1,14,30,0,0
2,0,0,0,0
3,0,0,0,0


In [158]:
# Train the KNN model
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train) 

C:\Users\natiami\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [159]:
# Accuracy of prediction on train data
y_pred = knn.predict(X_train)
print("Accuracy on train data ", 100*accuracy_score(y_pred, y_train ))

Accuracy on train data  77.1144278607


In [160]:
# Accuracy of prediction on test data
y_pred = knn.predict(X_test)
print("Accuracy on test data ", 100*accuracy_score(y_pred, y_test ))

Accuracy on test data  42.5287356322
